In [72]:
import pandas as pd

In [73]:
dfs = []
path = "./erasmus_mobility_stats_2014-2019/{}.csv"

# load all the .csv files, merge into a single dataframe

for year in range(2014, 2020):
    df = pd.read_csv(path.format(year), sep=";", parse_dates=["Mobility Start Month", "Mobility End Month"])
    dfs.append(df)

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [74]:
df = pd.concat(dfs, axis=0)

In [75]:
# Get rid of age out of bounds
df = df[df["Participant Age"].apply(lambda x: str(x).isnumeric() and int(x) < 100)]
df = df.astype({"Participant Age": int})

In [76]:
# rename columns for consistency (organisation -> organization)
df = df.rename(columns={"Sending Organisation Erasmus Code": "Sending Organization Erasmus Code", "Receiving Organisation Erasmus Code": "Receiving Organization Erasmus Code"})

In [77]:
# map the city names to upper for consistency
df["Sending City"] = df["Sending City"].apply(lambda x: str(x).upper())
df["Receiving City"] = df["Receiving City"].apply(lambda x: str(x).upper())

In [78]:
# parse the missing values
df["Participant Gender"] = df["Participant Gender"].apply(lambda x: None if x == "Undefined" in x else x)
df["Education Level"] = df["Education Level"].apply(lambda x: None if "Unknown" in x else x)
df["Field of Education"] = df["Field of Education"].apply(lambda x: None if "Unknown" in str(x) else x)
df["Receiving Organization Erasmus Code"] = df["Receiving Organization Erasmus Code"].apply(lambda x: None if x == "-" else x)
df["Sending Organization Erasmus Code"] = df["Sending Organization Erasmus Code"].apply(lambda x: None if x == "-" else x)

In [79]:
# parse columns into categories
category_columns = ['Academic Year', 'Mobility Start Month',
       'Mobility End Month', 'Mobility Duration', 'Activity (mob)',
       'Field of Education', 'Participant Nationality', 'Education Level',
       'Participant Gender', 'Participant Profile', 'Special Needs',
       'Fewer Opportunities', 'GroupLeader', 'Sending Country Code', 'Receiving Country Code']

df = df.astype({col: "category" for col in category_columns})

In [80]:
df.to_csv("erasmus-data.csv")

In [81]:
# make another dataframe just for the student exchanges
exchange_df = df[df["Activity (mob)"] == "Student mobility for studies between Programme Countries"]
exchange_df.to_csv("erasmus-only-student-exchange.csv")